In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os
import re
import ast

data_path = '/Users/amanda/Documents/Projects/insight/data/'

In [2]:
def convert_to_list(df,columns):

    df.fillna('', inplace=True)

    for col in columns:
        if isinstance(df[col][0], str):
            df[col] = [ast.literal_eval(s) for s in df[col]]

    return df

In [3]:
headers = {'User-Agent': 'Mozilla/5.0',
           'From': 'itchyandscratchy@gmail.com'
          }

In [4]:
# Load CIP codes and levels of education

code_df = pd.read_csv(os.path.join(data_path,'raw','cip-codes.csv'))
cip_codes = code_df['CIP Code'].str.strip("=").str.strip('"')
cip_list = cip_codes.tolist()

levels = ['LOS03','LOS04','LOS05','LOS06','LOS07']

In [5]:
# Try urls associated with each CIP code to find out if they exist
# Note this is a long process and only generally needs to be run once

url_list = []

for i in cip_list[0:10]:
    for j in levels:
    
        url = 'https://www.jobbank.gc.ca/studentdashboard/{}/{}'.format(i,j)
        print(url)
        request = requests.get(url, headers=headers, allow_redirects=False)
        if request.status_code == 200:
            url_list.append(url)

https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS03
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS04
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS05
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS06
https://www.jobbank.gc.ca/studentdashboard/15.1307/LOS07
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS03
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS04
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS05
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS06
https://www.jobbank.gc.ca/studentdashboard/61.2603/LOS07
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS03
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS04
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS05
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS06
https://www.jobbank.gc.ca/studentdashboard/32.0112/LOS07
https://www.jobbank.gc.ca/studentdashboard/52.0305/LOS03
https://www.jobbank.gc.ca/studentdashboard/52.0305/LOS04
https://www.jobbank.gc.ca/stude

In [6]:
# Save url_list to file so I can load it later

file_path = (os.path.join(data_path,'processed','url_list.txt'))

with open(file_path, 'w') as filehandle:
    for listitem in url_list:
        filehandle.write('%s\n' % listitem)

In [4]:
# Load url_list if not in memory

file_path = (os.path.join(data_path,'processed','url_list.txt'))

# define empty list
url_list = []

# open file and read the content in a list
with open(file_path, 'r') as filehandle:
    url_list = [current_place.rstrip() for current_place in filehandle.readlines()]

# Scrape education and associated jobs data

In [8]:
# Describe dataframe up front and add a row on each loop iteration to ensure 
# rows are always complete and there's no missing data resulting in problems
# at the end

jobs_df = pd.DataFrame({'edu_level':[],
                        'degree':[],
                        'description':[],
                        'employed':[],
                        'unemployed':[],
                        'med_earnings':[],
                        'range_earnings':[],
                        'related_field':[],
                        'unrelated_field':[],
                        'relation':[],
                        'top_jobs':[],
                        'top_job_links':[],
                        'job_percent':[],
                        'field_satisfaction':[]})

for url in url_list:
    
    try:
    
        with requests.get(url, timeout=5, headers=headers) as u:     
            soup = BeautifulSoup(u.content) 

            # the following is very specific to the website
            for a in soup.findAll('div', attrs={'class':'job-market'}):

                try:
                    # Add any interested attributes here
                    d_type = a.find('span', attrs={'class':'section-name'})
                    sec = a.find('span', attrs={'class':'heading-info'})
                    description = a.find('p', attrs={'class':'intro margin-top1 margin-bottom2'})

                    print(sec.text.strip()+':'+d_type.text.strip())

                    # Individual sections of the page described below
                    employment = a.find('div', attrs={'id':'employment_outcomes'})
                    p_employed = employment.find('p', attrs={'class':'stats_number_big'})
                    p_unemployed = employment.findAll('p', attrs={'class':'stats_number'})

                    earns = a.find('div', attrs={'class':'earning-section'})
                    med_earn = earns.find('p', attrs={'class':'stats_number_big'})
                    range_earn = earns.find('p', attrs={'class':'stats_number'})

                    related = a.find('div', attrs={'class':'related-section'})
                    close = related.find('p', attrs={'class':'stats_number_big'})
                    some_not = related.findAll('p', attrs={'class':'stats_number'}) 
                    how_related = [s.find_next_sibling('p') for s in some_not]

                    occupation = a.find('div', attrs={'class':'occupation-section'})
                    job_name = occupation.findAll('span', attrs={'class':'text-center'})
                    job_links = occupation.findAll('a', href=True)
                    job_percent = occupation.findAll('td', attrs={'headers':'top_ten_percent'})

                    satis = a.find('div', attrs={'class':'satisfaction-section'})
                    stud_again = satis.findAll('p', attrs={'class':'stats_number'})  

                except:               
                    continue

                # Organize attributes to append to dataframe
                if d_type is not None and sec is not None and description is not None:
                    degree_types = d_type.text.strip()
                    categories = sec.text.strip()
                    facts = description.text.strip()
                else:
                    continue

                if p_employed is not None and p_unemployed is not None:
                    stats_employed = p_employed.text.strip()
                    stats_unemployed = [s.text.strip() for s in p_unemployed]
                else:
                    stats_employed = np.NaN
                    stats_unemployed = np.NaN

                if med_earn is not None and range_earn is not None:
                    med_earnings = med_earn.text.strip()
                    range_earnings = range_earn.text.strip()
                else:
                    med_earnings = np.NaN
                    range_earnings = np.NaN

                if close:
                    related_fields = close.text.strip()
                else:
                    related_fields = np.NaN

                if some_not:
                    unrelated_fields = [s.text.strip() for s in some_not]
                    relation = [s.text.strip() for s in how_related]
                else:
                    unrelated_fields = np.NaN
                    relation = np.NaN

                if job_name is not None and job_links is not None and job_percent is not None:
                    if len(job_name) == len(job_links) == len(job_percent):
                        top_jobs = [s.text.strip() for s in job_name]
                        link_top_jobs = [s.get('href') for s in job_links]
                        per_top_jobs = [s.text.strip() for s in job_percent]
                else:
                    top_jobs = np.NaN
                    link_top_jobs = np.NaN
                    per_top_jobs = np.NaN

                if stud_again:
                    satisfaction = [s.text.strip() for s in stud_again]
                else:
                    satisfaction = np.NaN

                try:                
                    jobs_df = jobs_df.append({'edu_level':degree_types,
                                              'degree':categories,
                                              'description':facts,
                                              'employed':stats_employed,
                                              'unemployed':stats_unemployed,
                                              'med_earnings':med_earnings,
                                              'range_earnings':range_earnings,
                                              'related_field':related_fields,
                                              'unrelated_field':unrelated_fields,
                                              'relation':relation,
                                              'top_jobs':top_jobs,
                                              'top_job_links':link_top_jobs,
                                              'job_percent':per_top_jobs,
                                              'field_satisfaction':satisfaction}, ignore_index=True)

                except:
                    print('NOT APPENDED')
                    continue
    except:
        continue

Accounting and Business/Management:College/CEGEP
Accounting and Business/Management:Bachelor's degree
Accounting and Business/Management:Master's degree
Accounting and Computer Science:Other trades certificate or diploma
Accounting and Computer Science:College/CEGEP
Accounting and Computer Science:Bachelor's degree
Accounting and Computer Science:Master's degree
Accounting and Finance:College/CEGEP
Accounting and Finance:Bachelor's degree
Accounting and Finance:Master's degree
Accounting and Related Services, Other:College/CEGEP
Accounting and Related Services, Other:Bachelor's degree
Accounting and Related Services, Other:Master's degree
Accounting Technology/Technician and Bookkeeping:College/CEGEP
Accounting Technology/Technician and Bookkeeping:Bachelor's degree
Accounting Technology/Technician and Bookkeeping:Master's degree
Accounting:College/CEGEP
Accounting:Bachelor's degree
Accounting:Master's degree
Acoustics:College/CEGEP
Acoustics:Bachelor's degree
Acoustics:Master's degree

Agricultural Production Operations, General:Master's degree
Agricultural Production Operations, Other:Other trades certificate or diploma
Agricultural Production Operations, Other:College/CEGEP
Agricultural Production Operations, Other:Bachelor's degree
Agricultural Production Operations, Other:Master's degree
Agricultural Public Services, Other:Other trades certificate or diploma
Agricultural Public Services, Other:College/CEGEP
Agricultural Public Services, Other:Bachelor's degree
Agricultural Public Services, Other:Master's degree
Agricultural Teacher Education:College/CEGEP
Agricultural Teacher Education:Bachelor's degree
Agricultural Teacher Education:Master's degree
Agriculture, Agriculture Operations and Related Sciences, Other:Other trades certificate or diploma
Agriculture, Agriculture Operations and Related Sciences, Other:College/CEGEP
Agriculture, Agriculture Operations and Related Sciences, Other:Bachelor's degree
Agriculture, Agriculture Operations and Related Sciences, O

Apparel and Textiles, General:Bachelor's degree
Apparel and Textiles, General:Master's degree
Family and Consumer Sciences/Human Sciences, Other:Bachelor's degree
Family and Consumer Sciences/Human Sciences, Other:Master's degree
Appliance Installation and Repair Technology/Technician:Other trades certificate or diploma
Appliance Installation and Repair Technology/Technician:College/CEGEP
Applied Economics:College/CEGEP
Applied Economics:Bachelor's degree
Applied Economics:Master's degree
Applied Horticulture/Horticultural Business Services, Other:Other trades certificate or diploma
Applied Horticulture/Horticultural Business Services, Other:College/CEGEP
Applied Horticulture/Horticultural Business Services, Other:Bachelor's degree
Applied Horticulture/Horticultural Business Services, Other:Master's degree
Applied Horticulture/Horticulture Operations, General:Other trades certificate or diploma
Applied Horticulture/Horticulture Operations, General:College/CEGEP
Applied Horticulture/Hor

Baltic Studies:Master's degree
Banking and Financial Support Services:College/CEGEP
Banking and Financial Support Services:Bachelor's degree
Banking and Financial Support Services:Master's degree
Banking, Corporate, Finance and Securities Law (LLM, JSD/SJD):College/CEGEP
Banking, Corporate, Finance and Securities Law (LLM, JSD/SJD):Bachelor's degree
Banking, Corporate, Finance and Securities Law (LLM, JSD/SJD):Master's degree
Barbering/Barber:Other trades certificate or diploma
Barbering/Barber:College/CEGEP
Bartending/Bartender:Other trades certificate or diploma
Bartending/Bartender:College/CEGEP
Behavioural Sciences:College/CEGEP
Behavioural Sciences:Bachelor's degree
Behavioural Sciences:Master's degree
Bengali Language and Literature:College/CEGEP
Bengali Language and Literature:Bachelor's degree
Bengali Language and Literature:Master's degree
Bible/Biblical Studies:College/CEGEP
Bible/Biblical Studies:Bachelor's degree
Bible/Biblical Studies:Master's degree
Bicycle Mechanics and 

CAD/CADD Drafting and/or Design Technology/Technician:College/CEGEP
Canadian Government and Politics:College/CEGEP
Canadian Government and Politics:Bachelor's degree
Canadian Government and Politics:Master's degree
Canadian History:College/CEGEP
Canadian History:Bachelor's degree
Canadian History:Master's degree
Canadian Law/Legal Studies/Jurisprudence (LLM, MCJ, JSD/SJD):College/CEGEP
Canadian Law/Legal Studies/Jurisprudence (LLM, MCJ, JSD/SJD):Bachelor's degree
Canadian Law/Legal Studies/Jurisprudence (LLM, MCJ, JSD/SJD):Master's degree
Canadian Studies:College/CEGEP
Canadian Studies:Bachelor's degree
Canadian Studies:Master's degree
Cardiopulmonary Technology/Technologist:College/CEGEP
Cardiopulmonary Technology/Technologist:Bachelor's degree
Cardiopulmonary Technology/Technologist:Master's degree
Cardiovascular Science:College/CEGEP
Cardiovascular Science:Bachelor's degree
Cardiovascular Science:Master's degree
Cardiovascular Technology/Technologist:College/CEGEP
Cardiovascular Tec

...

Reproductive Biology:Bachelor's degree
Reproductive Biology:Master's degree
French as a Second Language:College/CEGEP
French as a Second Language:Bachelor's degree
French as a Second Language:Master's degree
French Language and Literature, General:College/CEGEP
French Language and Literature, General:Bachelor's degree
French Language and Literature, General:Master's degree
Second Language Learning, Other:College/CEGEP
Second Language Learning, Other:Bachelor's degree
Second Language Learning, Other:Master's degree
Technology Education/Industrial Arts Programs:College/CEGEP
French Language and Literature/Letters, Other:College/CEGEP
French Language and Literature/Letters, Other:Bachelor's degree
French Language and Literature/Letters, Other:Master's degree
Resort Management:College/CEGEP
Resort Management:Bachelor's degree
Resort Management:Master's degree
Respiratory Care Therapy/Therapist:College/CEGEP
Respiratory Care Therapy/Therapist:Bachelor's degree
Respiratory Care Therapy/Thera

Social Work, Other:Other trades certificate or diploma
Social Work, Other:College/CEGEP
Social Work, Other:Bachelor's degree
Social Work, Other:Master's degree
Social Work, General:Other trades certificate or diploma
Social Work, General:College/CEGEP
Social Work, General:Bachelor's degree
Social Work, General:Master's degree
Sociology:College/CEGEP
Sociology:Bachelor's degree
Sociology:Master's degree
Soil Chemistry and Physics:Other trades certificate or diploma
Soil Chemistry and Physics:College/CEGEP
Soil Chemistry and Physics:Bachelor's degree
Soil Chemistry and Physics:Master's degree
Soil Microbiology:Other trades certificate or diploma
Soil Microbiology:College/CEGEP
Soil Microbiology:Bachelor's degree
Soil Microbiology:Master's degree
Soil Science and Agronomy, General:Other trades certificate or diploma
Soil Science and Agronomy, General:College/CEGEP
Soil Science and Agronomy, General:Bachelor's degree
Soil Science and Agronomy, General:Master's degree
Soil Sciences, Other:O

Teacher Education and Professional Development, Specific Levels and Methods, Other:Master's degree
Teacher Education and Professional Development, Specific Subject Areas, Other:College/CEGEP
Teacher Education and Professional Development, Specific Subject Areas, Other:Bachelor's degree
Teacher Education and Professional Development, Specific Subject Areas, Other:Master's degree
Teacher Education, Multiple Levels:College/CEGEP
Teacher Education, Multiple Levels:Bachelor's degree
Teacher Education, Multiple Levels:Master's degree
Teaching Assistants/Aides, Other:College/CEGEP
Teaching Assistants/Aides, Other:Bachelor's degree
Teaching Assistants/Aides, Other:Master's degree
Teaching English as a Second or Foreign Language/ESL Language Instructor:College/CEGEP
Teaching English as a Second or Foreign Language/ESL Language Instructor:Bachelor's degree
Teaching English as a Second or Foreign Language/ESL Language Instructor:Master's degree
Teaching English or French as a Second or Foreign La

Vehicle Maintenance and Repair Technologies, Other:Other trades certificate or diploma
Vehicle Maintenance and Repair Technologies, Other:College/CEGEP
Veterinary Anatomy (Cert., MSc, PhD):Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Anatomy (Cert., MSc, PhD):Master's degree
Veterinary Anesthesiology Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Anesthesiology Residency Programs:Master's degree
Veterinary Biomedical and Clinical Sciences (Cert., MSc, PhD), Other:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Biomedical and Clinical Sciences (Cert., MSc, PhD), Other:Master's degree
Veterinary Dentistry Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Dentistry Residency Programs:Master's degree
Veterinary Dermatology Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Veterinary Dermatology Residency Program

Work and Family Studies:Other trades certificate or diploma
Work and Family Studies:College/CEGEP
Work and Family Studies:Bachelor's degree
Work and Family Studies:Master's degree
Writing:College/CEGEP
Yoga Teacher Training/Yoga Therapy:Bachelor's degree
Yoga Teacher Training/Yoga Therapy:Master's degree
Youth Ministry:College/CEGEP
Youth Ministry:Bachelor's degree
Youth Ministry:Master's degree
Youth Services/Administration:Other trades certificate or diploma
Youth Services/Administration:College/CEGEP
Youth Services/Administration:Bachelor's degree
Youth Services/Administration:Master's degree
Zoological Medicine Residency Programs:Degree in medicine, dentistry, veterinary medicine or optometry
Zoological Medicine Residency Programs:Master's degree
Zoology/Animal Biology, Other:College/CEGEP
Zoology/Animal Biology, Other:Bachelor's degree
Zoology/Animal Biology, Other:Master's degree
Zoology/Animal Biology, General:College/CEGEP
Zoology/Animal Biology, General:Bachelor's degree
Zoolo

In [9]:
# Check and clean the data before saving

edu_df_clean = jobs_df.dropna(subset=['edu_level','degree','description','top_jobs'])

edu_categories = [s.split('found under ')[1] for s in edu_df_clean['description']]
edu_categories = [s.strip('.') for s in edu_categories]
edu_df_clean['education_groups'] = edu_categories

# All education groups have the same recommended jobs, so we only need to keep one
education_df = edu_df_clean[['edu_level','degree','education_groups','top_jobs','job_percent','top_job_links']].drop_duplicates(subset =['education_groups','edu_level']) 
job_name_df = education_df[['edu_level','education_groups','top_jobs']].explode('top_jobs')
percent_df = education_df[['job_percent']].explode('job_percent')

link_df = education_df[['top_job_links']].explode('top_job_links')
link_df['top_job_links'].fillna('', inplace=True)

link_section = [x.split(';')[0] for x in link_df['top_job_links']]

job_name_df['job_percent'] = pd.to_numeric(percent_df['job_percent'].str.strip('%'),errors='coerce')
job_name_df['top_job_links'] = link_section
job_name_df['link'] = [x.replace('/marketreport/summary-occupation','') for x in link_section]
job_name_df['identifier'] = job_name_df['education_groups'].str.cat(job_name_df['edu_level'],sep=": ")

job_name_df.dropna(inplace=True)

In [10]:
# Save to file 

edu_df_clean.to_csv(os.path.join(data_path,'processed','jobbank-fields.csv'), index=False)
job_name_df.to_csv(os.path.join(data_path,'processed','education-to-job.csv'), index=False)

In [5]:
# Load csv if not in memory. Note that saving as csv flattens lists to strings.

job_name_df = pd.read_csv(os.path.join(data_path,'processed','education-to-job.csv'))
edu_df_load = pd.read_csv(os.path.join(data_path,'processed','jobbank-fields.csv'))

# Use eval to turn lists of strings back into lists
edu_df = convert_to_list(edu_df_load,['unemployed',
                                      'unrelated_field',
                                      'relation',
                                      'top_jobs',
                                      'top_job_links',
                                      'job_percent'])

In [6]:
# Load full job_url_list from previous notebook
# Note that previously I was using the output from the education scraping,
# but this is a much smaller list of jobs

file_path = (os.path.join(data_path,'processed','job_url_list.txt'))

# define empty list
url_end = []

# open file and read the content in a list
with open(file_path, 'r') as filehandle:
    url_end = [current_place.rstrip() for current_place in filehandle.readlines()]

# Scrape individual job data

In [9]:
# Go to each valid job link and scrape data there.

base_url = 'https://www.jobbank.gc.ca/marketreport/'

noc_list = []

overview_df = pd.DataFrame({'link':[],
                            'noc':[],
                            'job_title':[],
                            'salary':[]})

description_df = pd.DataFrame({'noc':[],
                               'description':[],
                               'duties':[],
                               'titles':[],
                               'related_occupations':[]})

regulation_df = pd.DataFrame({'noc':[],
                              'province':[],
                              'regulation':[]})

skills_df = pd.DataFrame({'noc':[],
                          'expertise':[],
                          'skills':[],
                          'knowledge':[]})


for link in url_end:
    
    try:
        # Main summary page
        url = base_url + 'summary-occupation' + link
        with requests.get(url, timeout=8, headers=headers) as u:     
            soup = BeautifulSoup(u.content) 

            # the following is very specific to the website
            a = soup.find('div', attrs={'class':'job-market'})
            
            # Add any interested attributes here
            title = a.find('span', attrs={'class':'heading-info'})
            intro = a.find('p', attrs={'class':'intro'})
            
            wages_section = a.find('a', attrs={'class':'wages-section'})
            salary = wages_section.find('p', attrs={'class':'section-value'})

            if title is not None and intro is not None:
                job_title = title.text.strip().replace(' in Canada','')
                job_intro = intro.text.strip()
                noc_text = job_intro.split('NOC ')[1]
                noc_code = 'NOC ' + noc_text.strip(').')
                
                print(noc_code + ': ' + job_title)
            else:
                continue

            if salary:
                job_salary = salary.text.strip()
            else:
                job_salary = 'NaN'

            overview_df = overview_df.append({'link': link,
                                              'noc': noc_code,
                                              'job_title': job_title,
                                              'salary': job_salary}, ignore_index=True)
            
    except:
        continue
        
    # For a given noc code, all the skills and other data is the same,
    # therefore we only need to collect for new noc codes
    
    if noc_code not in noc_list:
           
        try:    
            # Description page
            url = base_url + 'occupation' + link
            with requests.get(url, timeout=5, headers=headers) as u:     
                soup = BeautifulSoup(u.content) 

                desc_head = soup.find('h3', text = 'Description')
                if desc_head.findNext('p'):
                    description = desc_head.findNext('p')
                    job_description = description.text.strip()
                else:
                    description = 'NaN'

                job_duties = []
                duty_head = soup.find('h3', text = 'Job duties')
                duty_list = duty_head.findNext('li')
                for sib in duty_list.next_siblings:
                    if sib.name == 'li':
                        job_duties.append(sib.text.strip())
                    else:
                        break

                alt_titles = []
                title_head = soup.find('h3', text = 'Job titles').findNext('ul')
                title_bullets = title_head.findAll('li')
                if title_bullets:
                    alt_titles = [s.text.strip() for s in title_bullets]
                else:
                    alt_titles = 'NaN'

                rel_jobs = []
                related_head = soup.find('h3', text = 'Related occupations').findNext('ul')
                related_list = related_head.findAll('li')
                if related_list:
                    rel_jobs = [s.text.strip() for s in related_list]
                else:
                    rel_jobs = 'NaN'

                description_df = description_df.append({'noc': noc_code,
                                                        'description':job_description,
                                                        'duties':job_duties,
                                                        'titles':alt_titles,
                                                        'related_occupations':rel_jobs}, ignore_index=True)
                
                # Add noc code to the list so we don't scrape next time
                # only if we get this far
                noc_list.append(noc_code)
                
        except:
            pass

    # Outlook page
    #         url = base_url + 'outlook-occupation' + link
    #         with requests.get(url, timeout=5, headers=headers) as u:     
    #             soup = BeautifulSoup(u.content) 

    #             # the following is very specific to the website
    #             a = soup.find('div', attrs={'class':'job-market'})

    #             #try:
    #             # Add any interested attributes here
    #             prov_sec = a.findAll('td', attrs={'headers':'header1_outlook_prov'})
    #             for prov in prov_sec:
    #                 on_href = a.find('a', text = 'Ontario')
    #                 if on_href:
    #                     prospect = on_href.findNext('span', attr = {'class':'outlooknote'})
    #                     job_prospects = prospect.text.strip()
    #                 else:
    #                     job_prospects = 'NaN'


        try:

            # Skills page
            url = base_url + 'skills' + link
            with requests.get(url, timeout=5, headers=headers) as u:     
                soup = BeautifulSoup(u.content)

                #Expertise
                expert = []
                expert_head = soup.find('h2', text = 'Expertise').findNext('ul')
                expert_bullets = expert_head.findAll('li')
                if expert_bullets:
                    expert_list = [s.text.strip() for s in expert_bullets]
                else:
                    expert_list = 'NaN'

                #Skills
                skills = []
                skills_sec = soup.find('ul', attrs={'class':'sk'})
                skills_list = skills_sec.find('li')

                if skills_list:
                    skills_groups = [s.text.replace('\t','') for s in skills_list.find_next_siblings('li')]
                    for group in skills_groups:
                        skill_txt = group.split('\n')
                        skill_simple = [s.strip() for s in skill_txt if not 'Help - ' in s]
                        skill_simple = [s for s in skill_simple if s]
                        for skill in skill_simple[1:]:
                            skills.append(skill_simple[0].strip() + ' : ' + skill.strip())
                else:
                    skills = 'NaN'

                #Knowledge
                knowledge = []
                know_sec = skills_sec.findNext('ul', attrs={'class':'sk'})
                know_list = know_sec.find('li')

                if know_list:
                    know_groups = [s.text.replace('\t','') for s in know_list.find_next_siblings('li')]
                    for group in know_groups:
                        know_txt = group.split('\n')
                        know_simple = [s.strip() for s in know_txt if not 'Help - ' in s]
                        know_simple = [s for s in know_simple if s]
                        for know in know_simple[1:]:
                            knowledge.append(know_simple[0].strip() + ' : ' + know.strip())
                else:
                    know = 'NaN'

                skills_df = skills_df.append({'noc': noc_code,
                                              'expertise':expert_list,
                                              'skills':skills,
                                              'knowledge':knowledge}, ignore_index=True)

        except:
            pass
            
        try:
            
            # Certification page
            url = base_url + 'requirements' + link
            with requests.get(url, timeout=5, headers=headers) as u:     
                soup = BeautifulSoup(u.content) 
                province = []
                regulation = []
                for tr in soup.find_all('tr')[1:]:
                    tds = tr.find_all(recursive=False)
                    if tds:
                        province.append(tds[0].text.strip('\n \t'))
                        regulation.append(tds[2].text.strip('\n \t'))
                    else:
                        province = 'NaN'
                        regulation = 'NaN'

                regulation_df = regulation_df.append({'noc': noc_code,
                                                      'province':province,
                                                      'regulation':regulation}, ignore_index=True)                    
        except:
            pass



NOC 7205: Construction Painters Foreman/woman
NOC 7205: Asphalt Roofing Contractor
NOC 7205: Bricklaying And Masonry Contractor
NOC 7205: Contractor, Decorating
NOC 7205: Contractor, Drywall Application
NOC 7205: Contractor, Floor Covering
NOC 7205: Contractor, Glazing
NOC 7205: Contractor, Insulating
NOC 7205: Painting Contractor
NOC 7205: Contractor, Paperhanging
NOC 7205: Contractor, Plastering
NOC 7205: Roofing Contractor
NOC 7205: Contractor, Stonemasonry
NOC 7205: Contractor, Terrazzo Finishing
NOC 7205: Contractor, Tile, Terrazzo And Marble Setting
NOC 7205: Contractor, Tilesetting
NOC 7205: Contractor, Wood Shingle Roofing
NOC 7205: Customer Service Supervisor - Housing Construction
NOC 7205: Decorators Foreman/woman
NOC 7205: Eavestrough Installation Contractor
NOC 7205: Exterminating Service Supervisor
NOC 7205: Contractor, Fencing
NOC 7205: Floor Covering Foreman/woman
NOC 7205: Floor Covering Supervisor
NOC 7205: Asphalt Roofers Foreman/woman
NOC 7205: Boiler And Pipe Insul

NOC 7232: Precision Tool Maker
NOC 7232: Stamping Die Finisher
NOC 7232: Steel-rule Die Maker
NOC 7232: Template Maker - Tool And Die
NOC 7232: Tire Mould Repairer
NOC 7232: Tool And Die Maker
NOC 7232: Injection Moulding Tool And Die Maker
NOC 7232: Tool And Die Repairer
NOC 7232: Tool Fitter
NOC 7232: Tool Maintainer
NOC 7232: Toolmaker
NOC 7232: Tool Repairer
NOC 7232: Bench Die Fitter
NOC 7232: Forging Die Finisher
NOC 7232: Bench Jig Maker
NOC 7232: Mouldmaker, Metal
NOC 7232: Progressive Die Fitter
NOC 7232: Jig Maker - Die Casting And Plastic Moulding
NOC 7232: Injection Mouldmaker
NOC 7232: Tool And Die Preparer
NOC 7243: Apprentice Power System Electrician
NOC 7243: Maintainer, Voltage Regulator - Electrical Power System
NOC 7243: Power Station Electrician
NOC 7243: Electrical Powerhouse Electrician
NOC 7243: Power Station Electrician - Electrical Power System
NOC 7243: Relay Tester And Maintainer - Electrical Power System
NOC 7243: Relay Tester And Repairer - Electrical Power

NOC 9224: Foreman/woman, Mattress Manufacturing
NOC 9227: Baby Carriage Assembly Foreman/woman
NOC 9227: Foreman/woman, Bicycle Assembly
NOC 9227: Brush Making Foreman/woman
NOC 9227: Candle Making Foreman/woman
NOC 9227: Clock And Watch Inspection Foreman/woman
NOC 9227: Cooper Foreman/woman
NOC 9227: Door And Sash Assembly Foreman/woman
NOC 9227: Fibreglass Boat Assembly Foreman/woman
NOC 9227: Firearms Assembly Foreman/woman
NOC 9227: Lens Grinder And Polisher Foreman/woman
NOC 9227: Bench Assemblers Foreman/woman - Wood Products Manufacturing
NOC 9227: Foreman/woman, Clock And Watch Assembly
NOC 9227: Controlling And Recording Instruments Assembly Foreman/woman
NOC 9227: Cutlery Manufacturing Foreman/woman
NOC 9227: Jewellery And Silverware Manufacturing Worker Foreman/woman
NOC 9227: Foreman/woman, Jewellery Manufacturing
NOC 9227: Foreman/woman, Jewellery Repairing - Manufacturing
NOC 9227: Lamp Shade Fabrication Foreman/woman
NOC 9227: Foreman/woman, Millwork Assemblers - Wood M

NOC 7315: Aircraft Hydraulics Tester
NOC 7315: Aircraft Inspector
NOC 7315: Aircraft Inspector, Mechanical Systems
NOC 7315: Aircraft Maintenance Technician
NOC 7315: Aircraft Mechanical Systems Maintenance Technician
NOC 7315: Structures Aircraft Maintenance Engineer (AME)
NOC 7315: Apprentice Aircraft Mechanic, Mechanical Systems
NOC 7315: Aircraft Engine Overhaul Mechanic
NOC 7315: Aircraft Mechanic Experimental
NOC 7315: Flight Test Aircraft Mechanic
NOC 7315: Aircraft Mechanic, Mechanical Systems
NOC 7315: Aircraft Plumbing And Hydraulics Mechanic
NOC 7315: Aircraft Mechanical Systems Repair Inspector
NOC 7315: Aircraft Sheet Metal Technician
NOC 7315: Aircraft Skin Repairer
NOC 7315: Aircraft Systems Inspector
NOC 7315: Aircraft Mechanical Systems Technician
NOC 7315: Airframe Mechanic
NOC 7315: Rotorcraft Aircraft Maintenance Engineer (AME)
NOC 7315: Certified Aircraft Technician
NOC 7315: Aircraft Composite And Sheet Metal Repairer
NOC 7315: Engine Accessories Overhaul Mechanic

NOC 7332: Refrigerator Servicer
NOC 7332: Appliance Service Technician
NOC 7332: Appliance Repairer Apprentice
NOC 7332: Appliance Servicer
NOC 7333: Armature Repairman/woman
NOC 7333: Armature Winder Repairer
NOC 7333: Automotive-generator And Starter Repairer
NOC 7333: Circuit Breaker Mechanic
NOC 7333: Coil Connector Repairer
NOC 7333: Coil Winder And Repairer
NOC 7333: Electric Meter Repairer
NOC 7333: Electric Meter Repairer Apprentice
NOC 7333: Electric Meter Technician
NOC 7333: Electric Motor Repairer
NOC 7333: Electric Motor Repairer And Tester
NOC 7333: Electric Motor Winder-repairer
NOC 7333: Electrical Instrument Mechanic
NOC 7333: Electrical Mechanic
NOC 7333: Electrical Mechanic (except Avionics)
NOC 7333: Meter Repair Electrical Mechanic
NOC 7333: Switchgear Repair Electrical Mechanic
NOC 7333: Electrical Mechanic Trainee
NOC 7333: Electrical Transformer Repairer
NOC 7333: Generator Repairer
NOC 7333: Electrical Mechanics Group Leader
NOC 7333: High-voltage Power Transfo

...

NOC 4312: Structural Firefighter
NOC 4312: Training Firefighter
NOC 4312: Industrial Firefighter
NOC 4312: Fire Department Platoon Chief
NOC 4312: Firefighter-inspector
NOC 4312: Communication Service Firefighter
NOC 4312: Industrial Complex Firefighter
NOC 4312: Fire Prevention Officer
NOC 6346: Apprentice Embalmer
NOC 6346: Undertaker Assistant
NOC 6346: Funeral Director
NOC 6346: Mortician
NOC 6346: Assistant Funeral Director
NOC 6346: Funeral Home Director
NOC 6346: Funeral Services Director
NOC 6521: Reservation Agent - Travel Agency
NOC 6521: Travel Agent
NOC 6521: Agent, Travel Information
NOC 6521: Travel Consultant
NOC 6521: Counsellor, Travel Centre
NOC 6521: Travel Specialist
NOC 6521: Tourist Information Counsellor
NOC 6523: Airline Cargo Agent
NOC 6523: Station Agent - Airline
NOC 6523: Airline Baggage Agent
NOC 6523: Agent, Load Control - Airline
NOC 6523: Certified Load Planner - Airline
NOC 6523: Certified Load Control Agent - Airline
NOC 6523: Load Planner, Airline
NOC

NOC 7294: Maintenance Painter, Plant
NOC 7294: House Painter
NOC 6524: Ticket And Information Clerk (except Airline)
NOC 6524: Ticket Agent (except Airline)
NOC 6524: Ticket Clerk - Railway
NOC 6524: Agent, Tickets - Transit Station
NOC 6524: Transportation Agent (except Airline)
NOC 6524: Baggage Clerk (except Airline)
NOC 6524: Counter Clerk (except Airline)
NOC 6524: Client Service Representative - Bus Line
NOC 6524: Counter Sales Agent (except Airline)
NOC 6524: Sales Agent - Railway
NOC 6524: Ticket And Customer Service Clerk (except Airline)
NOC 6524: Agent, Counter Sales - Railway
NOC 6524: Cargo Customer Service Representative - Bus Line
NOC 6524: Cargo Customer Service Representative - Marine Shipping
NOC 6524: Cargo Customer Service Representative - Railway
NOC 6524: Agent, Ship Cargo
NOC 6524: Station Agent - Railway
NOC 6524: Freight Agent (except Airline)
NOC 6525: Hotel Front Desk Clerk
NOC 6525: Clerk, Hotel
NOC 6525: Night Clerk
NOC 6525: Reservations Clerk - Hotel
NOC 

NOC 7301: Foreman/woman, Heavy Agricultural Machinery Assembly
NOC 7301: Foreman/woman, Heavy Construction Equipment Assembly
NOC 7301: Foreman/woman, Heavy Equipment Assembly
NOC 7301: Foreman/woman, Heavy Farm Equipment Assembly
NOC 7301: Foreman/woman, Heavy Machinery Assembly
NOC 7301: Foreman/woman, Helicopter Maintenance - Mechanical Systems
NOC 7301: Foreman/woman, Industrial Machinery Manufacturing
NOC 7301: Foreman/woman, Industrial Mechanics
NOC 7301: Foreman/woman, Industrial, Farm And Construction Machinery Mechanics And Repairers
NOC 7301: Foreman/woman, Locomotive Inspectors
NOC 7301: Foreman/woman, Machine Builders
NOC 7301: Foreman/woman, Machine Fitters
NOC 7301: Foreman/woman, Manufacturing Machinery Assembly
NOC 7301: Foreman/woman, Marine Service - Engine Repair
NOC 7301: Foreman/woman, Mechanical Equipment Repair Inspectors
NOC 7301: Foreman/woman, Mechanical Inspectors
NOC 7301: Foreman/woman, Mechanical Equipment Repair Inspection
NOC 7301: Foreman/woman, Motor V

NOC 7303: Filmsetters Supervisor
NOC 7303: Photographic Processors Supervisor
NOC 7303: Film Processors Supervisor
NOC 7205: Foreman/woman, Bricklayer
NOC 7205: Bricklayer Supervisor
NOC 7205: Bricklaying Contractor
NOC 7205: Bridge Painters Foreman/woman
NOC 7205: Building Maintenance Supervisor
NOC 7205: Cement Finisher Foreman/woman
NOC 7205: Cement Finishing Contractor
NOC 7205: Asbestos Removal Supervisor
NOC 7441: Blinds Assembler-installer
NOC 1112: Change Management Consultant
NOC 0013: Chief Technology Officer (CTO)
NOC 2171: Cloud Architect — Information Technology (IT)
NOC 4151: Early Intervention Therapist
NOC 1226: Event Decorator
NOC 1111: Junior Accountant
NOC 6732: Kitchen Exhaust Cleaner
NOC 6221: Outside Sales Representative
NOC 7611: Renovation Helper
NOC 4214: Early Childhood Substitute Teacher
NOC 0651: Tattoo Studio Manager 
NOC 0711: Transportation Construction Manager 
NOC 0511: University Librarian
NOC 5131: Video Game Writer
NOC 6733: Construction Cleaner
NOC 

NOC 0912: Cellulosic Ethanol Production Plant Manager
NOC 0016: Energy Operations Vice-president
NOC 0912: Ethanol Operations Manager
NOC 0016: Plant Operations Vice-president 
NOC 0912: Production Co-ordinator – Energy
NOC 0912: Fuel Distribution Manager
NOC 0912: Biofuels Production Support Manager
NOC 9212: Hydro-electric Power Station Supervisor
NOC 0912: Hydro-electric Power Station Superintendent
NOC 0912: Hydro-electric Power Station Project Manager
NOC 0113: Category Purchasing Manager
NOC 0113: Strategic Sourcing Director
NOC 0731: Global Transportation Manager
NOC 1215: Load Out Logistics Supervisor
NOC 0714: Warehouse Operations Manager
NOC 0731: Global Logistics Manager
NOC 0731: Integrated Logistics Programs Director
NOC 0731: Logistics Analytics Manager
NOC 0731: Logistics Operations Director
NOC 1215: Logistics Solution Manager
NOC 1215: Logistics Team Leader
NOC 0016: Logistics Vice-president – Goods Production, Utilities, Transportation And Construction
NOC 0112: Benef

In [11]:
skills_df.dropna()

,noc,expertise,skills,knowledge
0,NOC 7205,"[Supervise workers and projects, Co-ordinate a...","[Service and Care : Protecting and Enforcing, ...","[Business, Finance and Management : Business ..."
1,NOC 7304,"[Supervise workers and projects, Ensure health...","[Service and Care : Protecting and Enforcing, ...","[Business, Finance and Management : Personnel ..."
2,NOC 7305,"[Supervise workers and projects, Train or arra...","[Service and Care : Protecting and Enforcing, ...","[Business, Finance and Management : Client Se..."
3,NOC 7232,[Compute dimensions and tolerances and set up ...,[Working with Technological Equipment and Mach...,[Engineering and Technology : Building and Con...
4,NOC 7243,[Install and maintain transformers and other p...,[Working with Technological Equipment and Mach...,[Engineering and Technology : Electricity (Ele...
...,...,...,...,...
418,NOC 6532,[Escort groups on outdoor sport and recreation...,"[Service and Care : Protecting and Enforcing, ...","[Business, Finance and Management : Client Se..."
419,NOC 6511,"[Advise on wine selection and serve wine, Liai...","[Service and Care : Serving Others, Management...","[Business, Finance and Management : Clerical, ..."
420,NOC 4422,[Escort detainees in transit and during tempor...,"[Service and Care : Protecting and Enforcing, ...",[Law and Public Safety : Public Safety and Sec...
421,NOC 4423,[Respond to citizen complaints concerning stra...,"[Service and Care : Serving Others, Service an...","[Law and Public Safety : Law, Government, and ..."


# Clean dataframes before saving

In [46]:
# Clean overview_df
overview_df['salary'] = [s.strip('$').replace('/hour','') for s in overview_df['salary']]
overview_df['salary'] = pd.to_numeric(overview_df['salary'],errors='coerce')

# Clean regulation_df
reg_df = regulation_df[['noc','province']].explode('province')
reg_explode = regulation_df[['regulation']].explode('regulation')

reg_df['regulation'] = reg_explode


In [44]:
overview_df.to_csv(os.path.join(data_path,'processed','job-overview.csv'), index=False)
description_df.to_csv(os.path.join(data_path,'processed','job-description.csv'), index=False)
reg_df.to_csv(os.path.join(data_path,'processed','job-regulation.csv'), index=False)
skills_df.to_csv(os.path.join(data_path,'processed','job-skills.csv'), index=False)

# Add noc_code to job_name_df for link between all DFs

In [13]:
# Load DFs if not in memory

overview_df = pd.read_csv(os.path.join(data_path,'processed','job-overview.csv'))
job_name_df = pd.read_csv(os.path.join(data_path,'processed','education-to-job.csv'))

In [47]:
job_noc_df = pd.merge(left=job_name_df, 
                      right=overview_df[['link','noc']].astype(str), 
                      how='left', 
                      left_on='link', 
                      right_on='link')

job_noc_df = job_noc_df.dropna()

In [48]:
# Save updated DF to file

job_noc_df.to_csv(os.path.join(data_path,'processed','education-to-job.csv'), index=False)


In [49]:
job_noc_df

,edu_level,education_groups,top_jobs,job_percent,top_job_links,link,identifier,noc
0,College/CEGEP,Accounting,Accounting and related clerks,11.99,/marketreport/summary-occupation/14122/ca,/14122/ca,Accounting: College/CEGEP,NOC 1431
1,College/CEGEP,Accounting,Financial auditors and accountants,11.23,/marketreport/summary-occupation/131/ca,/131/ca,Accounting: College/CEGEP,NOC 1111
2,College/CEGEP,Accounting,Accounting technicians and bookkeepers,10.46,/marketreport/summary-occupation/24500/ca,/24500/ca,Accounting: College/CEGEP,NOC 1311
3,College/CEGEP,Accounting,Administrative officers,3.92,/marketreport/summary-occupation/12462/ca,/12462/ca,Accounting: College/CEGEP,NOC 1221
4,College/CEGEP,Accounting,Retail salespersons,3.25,/marketreport/summary-occupation/20599/ca,/20599/ca,Accounting: College/CEGEP,NOC 6421
...,...,...,...,...,...,...,...,...
1935,Master's degree,Sociology,Administrative officers,2.63,/marketreport/summary-occupation/12462/ca,/12462/ca,Sociology: Master's degree,NOC 1221
1936,Master's degree,Sociology,Social and community service workers,2.63,/marketreport/summary-occupation/5112/ca,/5112/ca,Sociology: Master's degree,NOC 4212
1937,Master's degree,Sociology,Business development officers and marketing re...,2.63,/marketreport/summary-occupation/3916/ca,/3916/ca,Sociology: Master's degree,NOC 4163
1938,Master's degree,Sociology,University professors and lecturers,1.97,/marketreport/summary-occupation/4707/ca,/4707/ca,Sociology: Master's degree,NOC 4011
